In [122]:
import numpy as np
import pandas as pd
import copy
import glob

import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime
from plotly.subplots import make_subplots
pd.options.plotting.backend = "plotly"

In [123]:
def zs_data(start: str, end: str, src: str) -> pd.DataFrame:
    """Get data from """
    data_df = pd.read_csv(src, index_col=0)#, parse_dates=["Time"])

    freq='15min'
    intervals_per_hour = 4

    # temp bandaid to deal with substation data
    # data_df.index = pd.to_datetime(data_df.index, utc=True)#, format='mixed')

    # NOTE timestamps indicate start of an interval
    # Starting with 15 min data we'll set the time index to start at the beninning of this 
    # period and then trim off the first 5 min after ffill()
    tmp = pd.date_range(start=pd.to_datetime("2021-7-1 00:00").tz_localize("UTC"), 
                        end=pd.to_datetime("2023-06-30 23:45").tz_localize("UTC"), 
                        freq=freq)
    data_df.index = tmp

    # data_df = data_df.resample("5min").ffill()#.reset_index()

    # data_df.rename(columns={data_df.columns[0]: "network_load"}, inplace=True)

    new_df = data_df.loc[start:end]#[1:]

    # new_df.reset_index(inplace=True)

    # temp bandaid to deal with substation data
    # new_df = new_df.rename(columns={data_df.index.name:'settlementdate'})
    # new_df = new_df.rename(columns={'index':'settlementdate'})
    return new_df, intervals_per_hour

In [124]:
zs_df, intervals_per_hour = zs_data("2022-1-1 00:00", "2022-12-31 23:45",
    src = "data/evoenergy-zone-substation-report-2021-23.csv")

pop_SA3_df = pd.read_csv('data/SA3_population.csv', index_col=0)
pop_SA2_df = pd.read_csv('data/SA2_population.csv', index_col=0)
evo_cust_df = pd.read_csv('data/evo_customers.csv', index_col=0)

# http://www.bom.gov.au/climate/averages/tables/cw_070282.shtml
mean_3pm_temps = [26.9,26.4,23.5,19.1,14.9,11.4,10.6,12.6,15.1,18.3,22.0,24.8,18.8]
mean_temps_Bin = [20.495001,20.325001,17.85,13.48,9.469999,6.67,5.585,7.105,9.710001,12.985,15.895,18.775]

NextGen_solar = pd.read_csv('data/NextGen_solar.csv', index_col=0) 
NextGen_solar_normalised = NextGen_solar/NextGen_solar.min()
NextGen_solar_normalised.index = zs_df.index

SA3_dict = {'Gungahlin':['Gold Creek'],
            'Tuggeranong':['Wanniassa', 'Gilmore', 'Theodore'],
            'Woden Valley':['Woden', 'Wanniassa'],
            'Canberra East':['East Lake'],#, 'Fyshwick'],
            'South Canberra':['Telopea Park'],
            'Belconnen':['Belconnen', 'Latham'],
            'North Canberra':['Civic', 'City East'],
            'Weston Creek':['Woden'],
            'Molongolo':['Woden'],
            'Urriarra - Namadgi':['Tennent/Angle Crossing'],}

SA3_division = {'Woden Valley':[39238/(39238+24315+12102), 39238/(39238+88965)],
                'Tuggeranong':[88965/(39238+88965), 1, 1],
            'Weston Creek':[24315/(39238+24315+12102), ],
            'Molongolo':[12102/(39238+24315+12102)],
            'Gungahlin':[1],
            'Canberra East':[1,1],
            'South Canberra':[1],
            'Belconnen':[1,1],
            'North Canberra':[1,1]}

ACT_pop_total_current = 457565

zs_ratings = {'Gold Creek':[57,74], 
             'Gilmore':[45,62],  
             'Theodore':[45,62], 
            'Woden':[95,95], 
            'East Lake':[50,60], 
            'Fyshwick':[28,28], 
            'Telopea Park':[100,114], 
            'Belconnen':[55,74], 
            'Latham':[95,95], 
            'Civic':[110,114],
            'City East':[95,95],
            'Wanniassa':[95,95]} # Table 54  [Continuous, 2hr emergency] ratings

zs_total = [870,968]

ACT_solar_current = 448 # MW at 30/6/2024

ACT_vehicles = 318000 # 2021 data https://www.abs.gov.au/statistics/industry/tourism-and-transport/motor-vehicle-census-australia/latest-release

# 2,400kWh per year per car = 6.6kWh/day
# Round up to 7.4 for beautiful fit with Level 2 charger
# https://electricvehiclecouncil.com.au/docs/how-much-extra-electricity-will-i-use-in-my-home-when-i-buy-an-ev/
EV_daily_kWh = 6.6



In [125]:
# ng_demand = pd.DataFrame()
# ng_solar = pd.DataFrame()

# nextgen_files = glob.glob('/home/bjorn/Data/NextGen_csv/*.csv')
# for f in nextgen_files:
#     ng = pd.read_csv(f)
#     ng = ng.set_index('original index')
#     load = ng['load power (kW)'].to_frame(name=f)
#     solar_tmp = ng['solar power (kW)'].to_frame(name=f)
#     ng_demand = pd.concat([ng_demand,load], axis=1)
#     ng_solar = pd.concat([ng_solar,solar_tmp], axis=1)


# # ng_demand_av = ng_demand.mean(axis=1)
# ng_solar_av = ng_solar.mean(axis=1)

# # rearrange to match WW day of year
# # start_time = 1539435600 # 14/10/2018
# # end_time = 1531663200 #16/7/2018
# start_time = 1540990800 # 1/11/2018
# end_time = 1533045600 #1/8/2018
# end_2018_year = 1546261200
# start_2018_year = 1514725200
# # rearrange_year_demand = pd.concat([ng_demand_av.loc[start_time:end_2018_year],
# #     ng_demand_av.loc[start_2018_year:end_time]], axis=0)
# # rearrange_year_solar = pd.concat([ng_solar_av.loc[start_time:end_2018_year],
# #     ng_solar_av.loc[start_2018_year:end_time]], axis=0)

# # rearrange_year_demand = ng_demand_av.loc[start_2018_year:end_2018_year]
# rearrange_year_solar = ng_solar_av.loc[start_2018_year:end_2018_year]

# ng_solar_15min = rearrange_year_solar.groupby(np.arange(len(rearrange_year_solar.index)) // 3).mean()
# ng_solar_15min.index = zs_df.index
# ng_solar_15min.to_csv('NextGen_solar.csv', index=True) 

In [126]:
def forecast(evo_regions, evo_fractions, nu_days, start_day, zs_df, hw_times,
             current_households, future_households, mean_temps, 
             current_frac_gas, future_frac_gas, current_frac_heatpumps, future_frac_heatpumps,
             future_solar_MW, intervals_per_hour, EV_daytime_ratio, 
             EV_nighttime_ratio, EV_visiting_ratio, EV_times, future_EV_MWh_day, future_EV_ACT_MWh_day):
    current_frac_elec = 1 - current_frac_gas
    future_frac_elec = 1 - future_frac_gas
    current_frac_resist = 1 - current_frac_heatpumps
    future_frac_resist = 1 - future_frac_heatpumps

    date_dict = {}
    for i_d, day in enumerate(range(start_day,start_day+nu_days)):
        for e, zs in enumerate(evo_regions):
            if e == 0:
                current_day_zs_df = zs_df[zs][zs_df.index.dayofyear == day+1]*evo_fractions[e]
                current_day_zs_df.rename('Current', inplace=True)
            else:
                tmp_df = zs_df[zs][zs_df.index.dayofyear == day+1]*evo_fractions[e]
                tmp_df.rename('Current', inplace=True)
                # print(current_day_zs_df)
                # print(tmp_df)
                current_day_zs_df = current_day_zs_df.add(tmp_df)
                # print(current_day_zs_df)
        # current_day_zs_df.index = current_day_zs_df.index.strftime('%H:%M')
        # current_day_zs_df.index = current_day_zs_df.index.time
        # time_interval = current_day_zs_df.index[1] - current_day_zs_df.index[0]
        # print(current_day_zs_df)
        # print(repr(current_day_zs_df.index))

        date_dict[day] = current_day_zs_df.index[0].date()
        month = current_day_zs_df.index[0].month
        temp = mean_temps[month-1]

        current_gas_users = current_frac_gas*current_households
        future_elec_users = future_households*future_frac_elec
        
        pop_growth_factor = (future_households)/(current_households)
        
        # heating energy demand as function of temperature per household
        CBR_factor = 1.266
        hw_kWh_household_per_day = (10.295 - 0.2816*temp)*CBR_factor
        # Heat pump coefficient of performance as function of temperature
        heatpump_CoP = 2.9664 + 0.0703*temp

        current_hw_MWh_per_day_elec = (current_frac_resist*hw_kWh_household_per_day + \
                                       current_frac_heatpumps*hw_kWh_household_per_day/heatpump_CoP)/1000
        current_hw_MWh_per_day = current_frac_elec*current_households*current_hw_MWh_per_day_elec
        current_hw_MW = current_hw_MWh_per_day/24
        # print('current_hw_kWh', current_hw_MWh_per_day_elec*1000)
            
        curent_day_zs_sans_hw = current_day_zs_df - current_hw_MW
        curent_day_zs_sans_hw.rename('Current w/o hot water', inplace=True)


        ##################### 
        # Future hot water
        #####################
        future_solar_day_MW = future_solar_MW[future_solar_MW.index.dayofyear == day+1]

        # future_zs = current_day_zs_df*pop_growth_factor
        future_zs_sans_hw = curent_day_zs_sans_hw*pop_growth_factor - future_solar_day_MW['0']
        
        future_hw_resist_MWh = future_elec_users*future_frac_resist*hw_kWh_household_per_day/1000
        future_hw_heatpump_MWh = future_elec_users*future_frac_heatpumps*hw_kWh_household_per_day/heatpump_CoP/1000
        future_hw_MWh_total = future_hw_heatpump_MWh + future_hw_resist_MWh
        # print('future_hw_kWh', future_hw_MWh_total/future_elec_users*1000)

        for i, [timed_hw_start, timer_end] in enumerate(hw_times):
            if timed_hw_start < timer_end:
                time_mask = (current_day_zs_df.index.hour >= timed_hw_start) & \
                        (current_day_zs_df.index.hour < timer_end)
            else:
                time_mask = (current_day_zs_df.index.hour >= 0) & \
                        (current_day_zs_df.index.hour < timer_end) + \
                        (current_day_zs_df.index.hour >= timed_hw_start) & \
                        (current_day_zs_df.index.hour <= 24)

            timer_df = pd.Series(index=current_day_zs_df.index, data = 0)
            # timer_df[timer_df.index.hour > timed_hw_start & timer_df.index.hour < timer_end] == 1
            # print(timer_df.between_time(timed_hw_start,timer_end))
            timer_df[time_mask] = 1
            if i == 0:
                future_hw_MW = future_hw_heatpump_MWh / (np.sum(timer_df)/intervals_per_hour)
                timer_df *= future_hw_MW
                future_zs_with_hw = future_zs_sans_hw + timer_df
            else:
                future_hw_MW = future_hw_resist_MWh / (np.sum(timer_df)/intervals_per_hour)
                timer_df *= future_hw_MW
                future_zs_with_hw += timer_df
            future_zs_with_hw.rename('Future', inplace=True)
            # print(future_zs_with_hw)


        ##################### 
        # Future EVs
        #####################
        future_zs_with_EV = copy.deepcopy(future_zs_with_hw)
        for i, [timer_start, timer_end] in enumerate(EV_times):
            if timer_start < timer_end:
                time_mask = (current_day_zs_df.index.hour >= timer_start) & \
                        (current_day_zs_df.index.hour < timer_end)
            else:
                time_mask = (current_day_zs_df.index.hour >= 0) & \
                        (current_day_zs_df.index.hour < timer_end) + \
                        (current_day_zs_df.index.hour >= timer_start) & \
                        (current_day_zs_df.index.hour <= 24)

            timer_df = pd.Series(index=current_day_zs_df.index, data = 0)
            timer_df[time_mask] = 1
            if i == 0:
                EV_MW = (EV_daytime_ratio*future_EV_MWh_day + EV_visiting_ratio*future_EV_ACT_MWh_day) / (np.sum(timer_df)/intervals_per_hour)
            else:
                EV_MW = EV_nighttime_ratio*future_EV_MWh_day / (np.sum(timer_df)/intervals_per_hour)
            timer_df *= EV_MW
            future_zs_with_EV += timer_df
            future_zs_with_EV.rename('Future EVs', inplace=True)
            # print(future_zs_with_hw)




        ##################### 
        # Return values
        #####################
        plotting_df = pd.concat([current_day_zs_df, curent_day_zs_sans_hw, future_zs_with_hw,
                                 future_zs_with_EV],axis=1)

        current_day_zs_df.index = current_day_zs_df.index.time
        curent_day_zs_sans_hw.index = current_day_zs_df.index
        future_zs_sans_hw.index = current_day_zs_df.index
        future_zs_with_hw.index = current_day_zs_df.index
        future_zs_with_EV.index = current_day_zs_df.index

        if i_d == 0:
            many_day_df_current = current_day_zs_df.rename(date_dict[day], inplace=True).to_frame()
            many_day_df_current_sans = curent_day_zs_sans_hw.rename(date_dict[day], inplace=True).to_frame()
            many_day_df_future = future_zs_with_hw.rename(date_dict[day], inplace=True).to_frame()
            many_day_df_future_sans = future_zs_sans_hw.rename(date_dict[day], inplace=True).to_frame()
            many_day_df_future_EV = future_zs_with_EV.rename(date_dict[day], inplace=True).to_frame()
        else:
            many_day_df_current = many_day_df_current.join(current_day_zs_df.rename(date_dict[day]))
            many_day_df_current_sans = many_day_df_current_sans.join(curent_day_zs_sans_hw.rename(date_dict[day]))
            many_day_df_future = many_day_df_future.join(future_zs_with_hw.rename(date_dict[day]))
            many_day_df_future_sans = many_day_df_future_sans.join(future_zs_sans_hw.rename(date_dict[day]))
            many_day_df_future_EV = many_day_df_future_EV.join(future_zs_with_EV.rename(date_dict[day]))

    plotting_df_many = pd.concat([many_day_df_current.mean(axis=1), many_day_df_current_sans.mean(axis=1), 
                                  many_day_df_future_sans.mean(axis=1), many_day_df_future.mean(axis=1),
                                  many_day_df_future_EV.mean(axis=1)],axis=1)
    plotting_df_many.columns = ['Current','Current w/o hot water','Future w/o hot water','Future w hot water',
                                'Future w EVs']

    return many_day_df_current, many_day_df_current_sans, many_day_df_future, \
            many_day_df_future_sans, many_day_df_future_EV, plotting_df_many, plotting_df, future_hw_MWh_total

In [136]:
future_year = 2045
POE = 0.9
nu_days = 364
start_day = 0#5*30

# Hot water assumptions/variables
current_frac_gas = 0.45
future_frac_gas = 0.0
current_frac_heatpumps = 0.1
future_frac_heatpumps = 0.75

# Solar assumptions/variables
ACT_solar_future_total = 1000
timed_hw_start_heatpumps = 11
timed_hw_end_heatpumps = 15
timed_hw_start_resist = 22
timed_hw_end_resist = 6
# ACT_solar_future_total = 1500
# timed_hw_start_heatpumps = 10
# timed_hw_end_heatpumps = 16

# EV assumptions/variables
EV_daytime_ratio = 0.6
EV_nighttime_ratio = 1 - EV_daytime_ratio
EV_away_ratio = 0.3
EV_daytime_ratio *= (1-EV_away_ratio)
EV_nighttime_ratio *= (1-EV_away_ratio)
EV_visiting_ACT_ratio = 0.001 # fraction of ACT vehicles travelling to this area and charging during daytime
EV_daytime_start = 9
EV_daytime_end = 16
EV_nighttime_start = 22
EV_nighttime_end = 6

# Study region
SA_level = 3
SA_region = 'Belconnen'
# SA_region = 'Tuggeranong'
# SA_region = 'Gungahlin'
# SA_region = 'Woden Valley'
# SA_region = 'Canberra East'
# SA_region = 'South Canberra'
# SA_region = 'North Canberra'


current_households = 0
zs_combined_rating = 0
for e, r in enumerate(SA3_dict[SA_region]):
    zs_frac = SA3_division[SA_region][e]
    current_households += evo_cust_df.loc[r,"Evo resi customers"]*zs_frac
    zs_combined_rating += zs_ratings[r][0]*zs_frac # Continuous rating

if SA_level == 2:
    current_pop = pop_SA2_df.loc[SA_region,'2022']
    pop_per_household = current_pop/current_households
    # print(pop_per_household)
    future_households = pop_SA2_df.loc[SA_region,str(future_year)]/pop_per_household
    current_cars = ACT_vehicles*(current_pop/ACT_pop_total_current)
elif SA_level == 3:
    current_pop = pop_SA3_df.loc[SA_region,'2022']
    pop_per_household = current_pop/current_households
    future_households = pop_SA3_df.loc[SA_region,str(future_year)]/pop_per_household
    current_cars = ACT_vehicles*(current_pop/ACT_pop_total_current)

ACT_future_new_solar = ACT_solar_future_total - ACT_solar_current  # additions to current rooftop capacity across ACT
future_new_solar_capacity_MW = ACT_future_new_solar*zs_combined_rating/zs_total[0]
future_new_solar_MW = future_new_solar_capacity_MW*NextGen_solar_normalised

future_EV_MWh_day = current_cars*EV_daily_kWh/1000
future_EV_ACT_MWh_day = ACT_vehicles*EV_daily_kWh/1000
future_EV_MWh_region = future_EV_ACT_MWh_day*EV_visiting_ACT_ratio + future_EV_MWh_day *(1-EV_away_ratio)

# mean_temps = mean_3pm_temps
mean_temps = mean_temps_Bin
hw_times = [[timed_hw_start_heatpumps, timed_hw_end_heatpumps], [timed_hw_start_resist, timed_hw_end_resist]]
EV_times = [[EV_daytime_start, EV_daytime_end], [EV_nighttime_start, EV_nighttime_end]]

many_day_df_current, many_day_df_current_sans, many_day_df_future, \
    many_day_df_future_sans, many_day_df_future_EV, plotting_df_many, plotting_df, future_hw_MWh_total = \
        forecast(SA3_dict[SA_region], SA3_division[SA_region], nu_days, start_day, zs_df, hw_times,
             current_households, future_households, mean_temps,
             current_frac_gas, future_frac_gas, current_frac_heatpumps, future_frac_heatpumps, future_new_solar_MW,
             intervals_per_hour, EV_daytime_ratio, EV_nighttime_ratio, EV_visiting_ACT_ratio,
             EV_times, future_EV_MWh_day, future_EV_ACT_MWh_day)


In [137]:

# future_zs_with_hw.plot()
# many_day_df_current.plot()
# many_day_df_current.mean(axis=1).plot()
# plotting_df_many.plot()
fig = go.Figure(plotting_df_many.plot())
fig.update_yaxes(range = [0,zs_combined_rating])
fig.add_hline(y=zs_combined_rating, line_width=3, line_dash="dash", line_color="black")


In [138]:

# many_day_df_future.plot()

In [139]:
# Utilization
total_day_profile = many_day_df_current.mean(axis='columns')
total_day_profile_peak = total_day_profile.max()
current_tot_energy = total_day_profile.sum()/intervals_per_hour
total_day_profile = many_day_df_future_EV.mean(axis='columns')
total_day_profile_peak = total_day_profile.max()
future_tot_energy = total_day_profile.sum()/intervals_per_hour

POE_index = int(POE*nu_days)
current_peak_load_day = many_day_df_current.max().idxmax()
current_peak_load_MW = many_day_df_current[current_peak_load_day].max()
# peak_load_MW = many_day_df_future_EV.max(axis='columns')
future_peak_load_day = many_day_df_future_EV.max().idxmax()
future_peak_load_MW = many_day_df_future_EV[future_peak_load_day].max()
sorted_load_days = many_day_df_future_EV.max().sort_values(ascending=True)
high_load_days = sorted_load_days.iloc[POE_index:].index.to_numpy()

plot_overhead = 30 #MW
fig = go.Figure(many_day_df_future_EV[high_load_days].plot())
fig.update_yaxes(range = [0,zs_combined_rating+plot_overhead])
fig.add_hline(y=zs_combined_rating, line_width=3, line_dash="dash", line_color="black")


In [131]:
min_threshold_POE = many_day_df_future_EV[high_load_days].max().min()

print('Current total energy delivered = ', np.round(current_tot_energy,0), " MWh")
print('Current substation utilisation = ', np.round(100*current_tot_energy/(zs_combined_rating*24),0), " %")
print('Future total energy delivered = ', np.round(future_tot_energy,0), " MWh")
print('Future substation utilisation = ', np.round(100*future_tot_energy/(zs_combined_rating*24),0), " %")

print(f"Future threshold of {POE} POE = {np.round(min_threshold_POE,1)} MW")
print(f"Future max peak = {np.round(future_peak_load_MW,1)} MW")
print(f"Future HW demand = {np.round(future_hw_MWh_total,1)} MWh/day")
print(f"Future HW demand = {np.round((future_hw_MWh_total/future_households)*1000,1)} kWh/household/day")
print(f"Future HW demand = {np.round((future_hw_MWh_total/future_households)*250000*1000,1)} MWh for 250,000 households/day")
print(f"Future EV demand = {np.round(future_EV_MWh_region,1)} MWh/day")
print(f"Current households = {np.round((current_households)/1000,1)} k")
print(f"Future households = {np.round((future_households)/1000,1)} k")

Current total energy delivered =  155.0  MWh
Current substation utilisation =  13.0  %
Future total energy delivered =  249.0  MWh
Future substation utilisation =  21.0  %
Future threshold of 0.9 POE = 41.8 MW
Future max peak = 45.8 MW
Future HW demand = 0.5 MWh/day
Future HW demand = 2.7 kWh/household/day
Future HW demand = 673596.4 MWh for 250,000 households/day
Future EV demand = 217.9 MWh/day
Current households = 0.2 k
Future households = 0.2 k
